# Настройка окружения

In [ ]:
# библиотеки OpenCV и PyTorch
!pip install opencv-python torch

# библиотека cjm_byte_track для реализации алгоритма ByteTrack
!pip install cjm_byte_track

# библиотека effdet, которая предоставляет модели EfficientDet
!pip install effdet

# библиотека tabulate для форматирования данных в виде таблиц
!pip install tabulate

# библиотека tqdm для отображения прогресс-баров
!pip install tqdm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

# Импорт модулей

In [ ]:
import cv2
import numpy as np
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
from effdet.efficientdet import HeadNet
from cjm_byte_track.core import BYTETracker
from tabulate import tabulate
import time
from tqdm import tqdm

# Класс отслеживания объекта

In [ ]:
class CarTracker:
    def __init__(self, model_name, weights_url, device='cpu'):
        # Инициализация параметров трекера
        self.device = device # Выбор устройства
        self.model_name = model_name # Имя модели
        self.scores_per_frame = []  # Оценки детекций для каждого кадра
        self.processing_times = []  # Время обработки каждого кадра
        self.track_id_frames = {}  # Словарь для хранения кадров, на которых был обнаружен каждый трек
        self.current_frame_number = 0  # Номер текущего кадра

        # Задание конфигурации модели EfficientDet
        config = get_efficientdet_config(model_name)
        config.update({'num_classes': 90})
        net = EfficientDet(config, pretrained_backbone=True)
        net.class_net = HeadNet(config, num_outputs=config.num_classes)

        # Загрузка весов модели
        checkpoint = torch.hub.load_state_dict_from_url(weights_url)
        net.load_state_dict(checkpoint)

        # Инициализация модели для предсказаний
        self.model = DetBenchPredict(net).to(self.device).eval()
        # Инициализация трекера BYTETracker
        self.tracker = BYTETracker()

    def transform(self, image):
        # Получение оригинальных размеров изображения и их преобразование под вход модели с сохранением масштаба
        orig_h, orig_w = image.shape[:2]
        image_size = self.model.config.image_size
        scale = min(image_size[1] / orig_w, image_size[0] / orig_h)
        resized_w, resized_h = int(orig_w * scale), int(orig_h * scale)

        # Изменение размера изображения и вычисление отступов
        image = cv2.resize(image, (resized_w, resized_h))
        pad_top = (image_size[0] - resized_h) // 2
        pad_left = (image_size[1] - resized_w) // 2
        pad_bottom = image_size[0] - resized_h - pad_top
        pad_right = image_size[1] - resized_w - pad_left

        # Добавление отступов и нормализация изображения
        image = np.pad(image, ((pad_top, pad_bottom), (pad_left, pad_right), (0, 0)), mode='constant')
        image = image.astype(np.float32) / 255.0
        new_image = torch.from_numpy(image).permute(2, 0, 1).float()

        return new_image, scale, pad_top, pad_left

    def detect(self, image):
        # Запуск таймера для измерения времени обработки
        start_time = time.time()

        # Преобразование изображения
        transformed_image, scale, pad_top, pad_left = self.transform(image)
        transformed_image = transformed_image.unsqueeze(0).to(self.device)

        # Выполнение предсказаний
        with torch.no_grad():
            outputs = self.model(transformed_image)[0]

            # Извлечение рамок, оценок и меток классов
            bboxes, scores, labels = outputs[:, :4], outputs[:, 4], outputs[:, 5].int()
            car_class_id, score_threshold = 3, 0.5 # Установка класса "машина" и порога для фильтрации оценок
            valid_detections = (labels == car_class_id) & (scores > score_threshold) # Получение соответствующих детекций

            if valid_detections.any():
                bboxes, scores = bboxes[valid_detections], scores[valid_detections]

                max_score_idx = scores.argmax() # Находим индекс детекции с максимальной оценкой
                detections = torch.cat((bboxes[max_score_idx].unsqueeze(0), # Создаем для нее тензор с рамкой и оценкой
                                        scores[max_score_idx].unsqueeze(0).unsqueeze(0)), dim=1)

                orig_h, orig_w = image.shape[:2]
                # Преобразуем координаты рамок из масштаба модели в оригинальный масштаб изображения
                detections[:, [0, 2]] = (detections[:, [0, 2]] - pad_left) / scale
                detections[:, [1, 3]] = (detections[:, [1, 3]] - pad_top) / scale

                # Фильтруем рамки, чтобы они оставались в пределах изображения
                detections = detections[(detections[:, 0] >= 0) & (detections[:, 1] >= 0) &
                                        (detections[:, 2] <= orig_w) & (detections[:, 3] <= orig_h)]

                self.scores_per_frame.append(scores.max().item()) # Сохраняем максимальную оценку детекции для текущего кадра
            else:
                detections = torch.zeros((1, 5))

        # Остановка таймера и сохранение времени обработки
        self.processing_times.append(time.time() - start_time)

        return detections

    def track(self, frame):
        self.current_frame_number += 1 # Увеличение номера текущего кадра
        bboxes_and_scores = self.detect(frame) # Получение результатов детекции на текущем кадре

        if bboxes_and_scores.shape[0] == 0:
            return [], []

        # Обновление трекера с новыми детекциями
        img_info = frame.shape[:2]
        bboxes_and_scores = bboxes_and_scores.cpu().numpy()
        online_targets = self.tracker.update(bboxes_and_scores, img_info, img_info)
        tracked_bboxes = [track.tlwh for track in online_targets]
        track_ids = [track.track_id for track in online_targets]

        # Обновление информации о треках
        for track_id in track_ids:
            if track_id not in self.track_id_frames:
                self.track_id_frames[track_id] = set()
            self.track_id_frames[track_id].add(self.current_frame_number)

        return tracked_bboxes, track_ids

    def get_number_of_tracks(self):
        # Возвращение количества треков
        return len(self.track_id_frames)

    def get_total_track_length(self):
        # Возвращение суммарной длины трека
        return sum(len(frames) for frames in self.track_id_frames.values())

# Отрисовка рамки

In [ ]:
def draw_tracks(frame, tracked_bboxes, track_ids):
    for bbox, track_id in zip(tracked_bboxes, track_ids):
        x1, y1, w, h = [int(i) for i in bbox]
        x2, y2 = x1 + w, y1 + h
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    return frame

# Обработка видео

In [ ]:
def process_video(input_path, output_path, tracker):
    cap = cv2.VideoCapture(input_path) # Открытие видеофайла для чтения
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Получение общего количества кадров

    # Инициализация кодека и видеофайла для записи
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    frame_count = 0  # Счетчик обработанных кадров

    # Отображение прогресса обработки видео
    with tqdm(total=total_frames, desc="Обработка видео", unit="кадр") as pbar:
        while cap.isOpened():
            # Чтение следующего кадра из видео
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            tracked_bboxes, track_ids = tracker.track(frame) # Выполнение трекинга объектов на текущем кадре
            frame = draw_tracks(frame, tracked_bboxes, track_ids) # Отрисовка рамок и идентификаторов треков на кадре
            out.write(frame) # Запись обработанного кадра в выходной видеофайл
            pbar.update(1) # Обновление индикатора прогресса

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Освобождение ресурсов
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Вычисление средней оценки детекций
    avg_score = round(np.mean(tracker.scores_per_frame), 3) if tracker.scores_per_frame else 0

    # Вычисление FPS количества кадров в секунду
    fps = frame_count / (sum(tracker.processing_times) + 1e-6)

    number_of_tracks = tracker.get_number_of_tracks() # Получение количества треков
    total_track_length = tracker.get_total_track_length() # Получение суммарной длины трека

    # Формирование строки с соотношением суммарной длины треков к общему количеству кадров
    track_length_ratio = f"{total_track_length}/{total_frames}"

    # Возврат результатов обработки видео
    return [
        tracker.model_name,
        avg_score,
        f"{fps:.2f}",
        number_of_tracks,
        track_length_ratio
    ]

# Тесты

# Папка с входными видеоматериалами и результатами тестов
https://drive.google.com/drive/folders/1mnk72wG1V98_XhoU73x23MU63oKKGC7M?usp=sharing

# red_car_240x426.mp4


In [ ]:
input_video = '/content/red_car_240x426.mp4'
output_video = '/content/lite1_red_car_240x426.mp4'

model_name = 'tf_efficientdet_lite1'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite1-6dc7ab30.pth'
tracker = CarTracker(model_name, weights_url)

results1 = process_video(input_video, output_video, tracker)
table1 = [results1]

# Создание таблицы с результатами
headers = ["Модель", "Средняя оценка детекций","FPS", "Количество треков", "Суммарная длина трека в кадрах"]
print("\n" + tabulate(table1, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 215/215 [00:57<00:00,  3.71кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1 |          0.609          | 3.79 |         1         |            175/215             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite2_red_car_240x426.mp4'

model_name = 'tf_efficientdet_lite2'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite2-86c5d55b.pth'
tracker = CarTracker(model_name, weights_url)

results2 = process_video(input_video, output_video, tracker)
table2 = [results2]

print("\n" + tabulate(table2, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 215/215 [01:28<00:00,  2.43кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite2 |          0.653          | 2.46 |         1         |            169/215             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3_red_car_240x426.mp4'

model_name = 'tf_efficientdet_lite3'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3-506852cb.pth'
tracker = CarTracker(model_name, weights_url)

results3 = process_video(input_video, output_video, tracker)
table3 = [results3]

print("\n" + tabulate(table3, headers, tablefmt="pretty"))

model.safetensors:   0%|          | 0.00/33.1M [00:00<?, ?B/s]

Downloading: "https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3-506852cb.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientdet_lite3-506852cb.pth
100%|██████████| 32.4M/32.4M [00:01<00:00, 33.3MB/s]
Обработка видео: 100%|██████████| 215/215 [02:56<00:00,  1.22кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3 |          0.724          | 1.23 |         1         |            208/215             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3x_red_car_240x426.mp4'

model_name = 'tf_efficientdet_lite3x'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3x-8404c57b.pth'
tracker = CarTracker(model_name, weights_url)

results4 = process_video(input_video, output_video, tracker)
table4 = [results4]

print("\n" + tabulate(table4, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 215/215 [04:56<00:00,  1.38s/кадр]


+------------------------+-------------------------+------+-------------------+--------------------------------+
|         Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+------------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3x |          0.745          | 0.73 |         1         |            214/215             |
+------------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
table = []
table.append(results1)
table.append(results2)
table.append(results3)

print(tabulate(table, headers, tablefmt="pretty"))

+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1 |          0.609          | 3.79 |         1         |            175/215             |
| tf_efficientdet_lite2 |          0.653          | 2.46 |         1         |            169/215             |
| tf_efficientdet_lite3 |          0.724          | 1.23 |         1         |            208/215             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


# black_car_240x426.mp4


In [ ]:
input_video = '/content/black_car_240x426.mp4'
output_video = '/content/lite1_black_car_240x426.mp4'

model_name = 'tf_efficientdet_lite1'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite1-6dc7ab30.pth'
tracker = CarTracker(model_name, weights_url)

results1 = process_video(input_video, output_video, tracker)
table1 = [results1]

headers = ["Модель", "Средняя оценка детекций","FPS", "Количество треков", "Суммарная длина трека в кадрах"]
print("\n" + tabulate(table1, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 452/452 [02:12<00:00,  3.40кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1 |          0.605          | 3.46 |         1         |            169/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite2_black_car_240x426.mp4'

model_name = 'tf_efficientdet_lite2'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite2-86c5d55b.pth'
tracker = CarTracker(model_name, weights_url)

results2 = process_video(input_video, output_video, tracker)
table2 = [results2]

print("\n" + tabulate(table2, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 452/452 [03:11<00:00,  2.36кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite2 |          0.716          | 2.38 |         1         |            162/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3_black_car_240x426.mp4'

model_name = 'tf_efficientdet_lite3'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3-506852cb.pth'
tracker = CarTracker(model_name, weights_url)

results3 = process_video(input_video, output_video, tracker)
table3 = [results3]

print("\n" + tabulate(table3, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 452/452 [05:58<00:00,  1.26кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3 |          0.797          | 1.27 |         1         |            350/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3x_black_car_240x426.mp4'

model_name = 'tf_efficientdet_lite3x'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3x-8404c57b.pth'
tracker = CarTracker(model_name, weights_url)

results4 = process_video(input_video, output_video, tracker)
table4 = [results4]

print("\n" + tabulate(table4, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 452/452 [10:32<00:00,  1.40s/кадр]


+------------------------+-------------------------+------+-------------------+--------------------------------+
|         Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+------------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3x |          0.802          | 0.72 |         1         |            452/452             |
+------------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
table = []
table.append(results1)
table.append(results2)
table.append(results3)

print(tabulate(table, headers, tablefmt="pretty"))

+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1 |          0.605          | 3.46 |         1         |            169/452             |
| tf_efficientdet_lite2 |          0.716          | 2.38 |         1         |            162/452             |
| tf_efficientdet_lite3 |          0.797          | 1.27 |         1         |            350/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


# red_car_360x640.mp4


In [ ]:
input_video = '/content/red_car_360x640.mp4'
output_video = '/content/lite1_red_car_360x640.mp4'

model_name = 'tf_efficientdet_lite1'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite1-6dc7ab30.pth'
tracker = CarTracker(model_name, weights_url)

results1 = process_video(input_video, output_video, tracker)
table1 = [results1]

headers = ["Модель", "Средняя оценка детекций","FPS", "Количество треков", "Суммарная длина трека в кадрах"]
print("\n" + tabulate(table1, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 215/215 [00:56<00:00,  3.83кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1 |          0.617          | 3.95 |         1         |            170/215             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite2_red_car_360x640.mp4'

model_name = 'tf_efficientdet_lite2'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite2-86c5d55b.pth'
tracker = CarTracker(model_name, weights_url)

results2 = process_video(input_video, output_video, tracker)
table2 = [results2]

print("\n" + tabulate(table2, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 215/215 [01:33<00:00,  2.30кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite2 |          0.655          | 2.35 |         1         |            169/215             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3_red_car_360x640.mp4'

model_name = 'tf_efficientdet_lite3'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3-506852cb.pth'
tracker = CarTracker(model_name, weights_url)

results3 = process_video(input_video, output_video, tracker)
table3 = [results3]

print("\n" + tabulate(table3, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 215/215 [03:04<00:00,  1.17кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3 |          0.736          | 1.18 |         1         |            210/215             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3x_red_car_360x640.mp4'

model_name = 'tf_efficientdet_lite3x'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3x-8404c57b.pth'
tracker = CarTracker(model_name, weights_url)

results4 = process_video(input_video, output_video, tracker)
table4 = [results4]

print("\n" + tabulate(table4, headers, tablefmt="pretty"))

Downloading: "https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3x-8404c57b.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientdet_lite3x-8404c57b.pth
100%|██████████| 36.0M/36.0M [00:01<00:00, 35.5MB/s]
Обработка видео: 100%|██████████| 215/215 [04:59<00:00,  1.39s/кадр]


+------------------------+-------------------------+------+-------------------+--------------------------------+
|         Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+------------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3x |          0.755          | 0.72 |         1         |            215/215             |
+------------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite4_red_car_360x640.mp4'

model_name = 'tf_efficientdet_lite4'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite4-391ddabc.pth'
tracker = CarTracker(model_name, weights_url)

results5 = process_video(input_video, output_video, tracker)
table5 = [results4]

print("\n" + tabulate(table5, headers, tablefmt="pretty"))

In [ ]:
table = []
table.append(results1)
table.append(results2)
table.append(results3)
table.append(results4)

print(tabulate(table, headers, tablefmt="pretty"))

+------------------------+-------------------------+------+-------------------+--------------------------------+
|         Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+------------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1  |          0.617          | 3.95 |         1         |            170/215             |
| tf_efficientdet_lite2  |          0.655          | 2.35 |         1         |            169/215             |
| tf_efficientdet_lite3  |          0.736          | 1.18 |         1         |            210/215             |
| tf_efficientdet_lite3x |          0.755          | 0.72 |         1         |            215/215             |
+------------------------+-------------------------+------+-------------------+--------------------------------+


# black_car_360x640.mp4


In [ ]:
input_video = '/content/black_car_360x640.mp4'
output_video = '/content/lite1_black_car_360x640.mp4'

model_name = 'tf_efficientdet_lite1'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite1-6dc7ab30.pth'
tracker = CarTracker(model_name, weights_url)

results1 = process_video(input_video, output_video, tracker)
table1 = [results1]

headers = ["Модель", "Средняя оценка детекций","FPS", "Количество треков", "Суммарная длина трека в кадрах"]
print("\n" + tabulate(table1, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 452/452 [01:52<00:00,  4.02кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1 |          0.587          | 4.13 |         1         |            143/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite2_black_car_360x640.mp4'

model_name = 'tf_efficientdet_lite2'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite2-86c5d55b.pth'
tracker = CarTracker(model_name, weights_url)

results2 = process_video(input_video, output_video, tracker)
table2 = [results2]

print("\n" + tabulate(table2, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 452/452 [03:06<00:00,  2.43кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite2 |          0.724          | 2.47 |         1         |            146/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3_black_car_360x640.mp4'

model_name = 'tf_efficientdet_lite3'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3-506852cb.pth'
tracker = CarTracker(model_name, weights_url)

results3 = process_video(input_video, output_video, tracker)
table3 = [results3]

print("\n" + tabulate(table3, headers, tablefmt="pretty"))

Обработка видео: 100%|██████████| 452/452 [05:53<00:00,  1.28кадр/s]


+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3 |           0.8           | 1.29 |         1         |            394/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
output_video = '/content/lite3x_black_car_360x640.mp4'

model_name = 'tf_efficientdet_lite3x'
weights_url = 'https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3x-8404c57b.pth'
tracker = CarTracker(model_name, weights_url)

results4 = process_video(input_video, output_video, tracker)
table4 = [results4]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/33.1M [00:00<?, ?B/s]

Downloading: "https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_lite3x-8404c57b.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientdet_lite3x-8404c57b.pth
100%|██████████| 36.0M/36.0M [00:00<00:00, 52.3MB/s]
Обработка видео: 100%|██████████| 452/452 [10:10<00:00,  1.35s/кадр]


NameError: name 'headers' is not defined

In [ ]:
headers = ["Модель", "Средняя оценка детекций","FPS", "Количество треков", "Суммарная длина трека в кадрах"]
print("\n" + tabulate(table4, headers, tablefmt="pretty"))


+------------------------+-------------------------+------+-------------------+--------------------------------+
|         Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+------------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite3x |          0.838          | 0.74 |         1         |            452/452             |
+------------------------+-------------------------+------+-------------------+--------------------------------+


In [ ]:
table = []
table.append(results1)
table.append(results2)
table.append(results3)

print(tabulate(table, headers, tablefmt="pretty"))

+-----------------------+-------------------------+------+-------------------+--------------------------------+
|        Модель         | Средняя оценка детекций | FPS  | Количество треков | Суммарная длина трека в кадрах |
+-----------------------+-------------------------+------+-------------------+--------------------------------+
| tf_efficientdet_lite1 |          0.587          | 4.13 |         1         |            143/452             |
| tf_efficientdet_lite2 |          0.724          | 2.47 |         1         |            146/452             |
| tf_efficientdet_lite3 |           0.8           | 1.29 |         1         |            394/452             |
+-----------------------+-------------------------+------+-------------------+--------------------------------+


# Использованные источники

*   https://blog.roboflow.com/top-object-tracking-software/
*   https://www.hitechbpo.com/blog/top-object-detection-models.php
*   https://vc.ru/dev/656805-top-5-metodov-trekinga-obektov?ysclid=lysmi6hk4w673224911
*   https://habr.com/ru/articles/503766/
*   https://github.com/ifzhang/ByteTrack
   *   https://arxiv.org/pdf/2110.06864
*   https://github.com/rwightman/efficientdet-pytorch
    *   https://arxiv.org/pdf/1911.09070
*   https://medium.com/tech-blogs-by-nest-digital/object-tracking-object-detection-tracking-using-bytetrack-0aafe924d292
*   https://www.ikomia.ai/blog/bytetrack-multi-object-tracking#:~:text=At%20its%20core%2C%20ByteTrack%20transcends%20traditional%20tracking%20methods,handling%20scenarios%20that%20would%20confound%20conventional%20tracking%20systems
*   https://blog.roboflow.com/yolov3-versus-efficientdet-for-state-of-the-art-object-detection/
*   https://medium.com/suitable-ryans-thoughts/yolov8-vs-efficientdet-a-deep-dive-into-the-future-of-real-time-object-detection-code-at-bottom-86513de2f69d

